In [3]:
from sklearn.datasets import make_classification

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
import pandas as pd
from sklearn.metrics import make_scorer, fbeta_score



In [4]:
df = pd.read_csv('../data/GeneralDatensatz18-21ohneGeo-mitLockdown_mitCorona.csv', sep=';')

X=df[['UMONAT','USTUNDE','UWOCHENTAG','UART','USTRZUSTAND','BEZ','UTYP1','ULICHTVERH','IstRad','IstPKW','IstFuss','IstKrad','IstGkfz','IstSonstige', 'LOCKDOWN', 'COVID']]
#für tödliche Unfälle

#für tödliche und schwere vs. leichte Unfälle  -> 1 ist schwer oder tödlich, 0 ist leicht
y = df['UKATEGORIE'].isin([1, 2]).astype(int)

In [5]:
beta = 2
fbeta_scorer= make_scorer(fbeta_score, beta=beta)

In [6]:
param_dist = {
    'n_estimators': list(range(100, 500, 4)),
    'max_depth': list(range(1, 18)),
    'min_samples_split': list(range(100, 500, 4)),
    'min_samples_leaf': list(range(2, 100, 3)),
    'max_features': ['auto', 'sqrt', 'log2',8, 10, 12, 14, 16],
    
    'class_weight': [{0:1, 1: 9}]
}

stratified_kfold = StratifiedKFold(n_splits=5)
rf = RandomForestClassifier()
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=100, cv=stratified_kfold, scoring=fbeta_scorer)
random_search.fit(X, y)
best_params = random_search.best_params_

/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
85 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
85 fits failed with the following error:
Traceback (most recent call last):
  File "/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_param

In [7]:
best_params

{'n_estimators': 344,
 'min_samples_split': 400,
 'min_samples_leaf': 8,
 'max_features': 'log2',
 'max_depth': 15,
 'class_weight': {0: 1, 1: 9}}